In [1]:
%load_ext autoreload
%autoreload 2
from labeling import *
from CO2_identify import resize3d, mute_top
from itertools import product
from collections import OrderedDict
from scipy.ndimage import gaussian_filter
from sklearn.neighbors import NearestNeighbors
import re
import pandas as pd
import json
import torch
import glob
import shutil

In [2]:
fn = '../define_path.txt'
with open(fn) as f:
    lines = f.readlines()
for idx, line in enumerate(lines):
    if idx == 1:
        dir_co2 = line.split('=')[1][:-1]
    if idx == 3:
        dir_grid = line.split('=')[1][:-1]
    if idx == 5:
        outpath0 = line.split('=')[1][:-1]
    if idx == 7:
        outpath = line.split('=')[1][:-1]

# Training dataset

##  Basic pathes for baseline data and time-lapse data

In [3]:
# file pathes
blfn = f'{dir_co2}/94p10/2010 processing/data/94p10nea.sgy' # baseline data processed in 2010
blfn2 = f'{dir_co2}/94p01/2001 processing/data/94p01nea.sgy' # baseline data processed in 2001
tlfn = f'{dir_co2}/10p10/2010 processing/data/10p10nea.sgy' # timelapse (2010) data processed in 2010
tlfn2 = f'{dir_co2}/10p11/2011 processing/data/10p11nea' # timelapse (2011) data processed in 2011
mkfn = f'../resources/label/masks.dat' # CO2 masks interpreted from 1994 and 2010 data processed in 2010

## Load data

In [4]:
# load the data head
D0 = dataload(fn=blfn)
D0n = dataload(fn=blfn2)
Dt = dataload(fn=tlfn)
Dtn = dataload(fn=tlfn2)

Total trace number: 150909
Sample number along each trace: 1001
Sampling interval along each trace: 0.002 s
data arrangement: 269 (number of ensembles) x 561 (trace number per ensemble) x 1001 (sample number per trace)
Total trace number: 166747
Sample number along each trace: 1001
Sampling interval along each trace: 0.002 s
data arrangement: 287 (number of ensembles) x 581 (trace number per ensemble) x 1001 (sample number per trace)
Total trace number: 150909
Sample number along each trace: 1001
Sampling interval along each trace: 0.002 s
data arrangement: 269 (number of ensembles) x 561 (trace number per ensemble) x 1001 (sample number per trace)
Total trace number: 242136
Sample number along each trace: 1001
Sampling interval along each trace: 0.002 s
data arrangement: 216 (number of ensembles) x 1121 (trace number per ensemble) x 1001 (sample number per trace)


In [5]:
# mask dimension
DD = (Dt.nx,Dt.ny,Dt.nt)

In [6]:
# get the entire 3D data from the three data volume
d0,xd0,yd0,td0 = D0.getdata()
d0n,xdn,ydn,_ = D0n.getdata()
dt,xd,yd,td = Dt.getdata()
dtn,xdt,ydt,tdt = Dtn.getdata()

## Preprocess d0n and dtn to make them in the same dimension as d0 and dt

In [7]:
# interpolate d0n to the same dimension as dt(d0)
X = np.stack((xd.flatten(),yd.flatten()),axis=1)
Xn = np.stack((xdn.flatten(),ydn.flatten()),axis=1)
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(Xn)
distances, ind = nbrs.kneighbors(X)
sub = ind2sub(xdn.shape,ind)
d0i = d0n[sub[0],sub[1],:]
d0i = np.reshape(d0i,d0.shape)

In [8]:
# interpolate dtn the same dimension as dt(d0)
# downsample along xline direction
dtn1 = np.pad(dtn[:,1:-3:2,:],((0,0),(2,0),(0,0)),'edge')
# calculate the coefficient for linear interpolation
xdh,xdth = xd[:,0],xdt[:,0]
X = np.stack((xdh,np.zeros(Dt.nx)),axis=1)
Xt = np.stack((xdth,np.zeros(Dtn.nx)),axis=1)
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(Xt)
distances, ind = nbrs.kneighbors(X)
coe = np.flip(distances,axis=1)/np.expand_dims(np.sum(distances,axis=1),1)
# linear interpolate along inline direction
dti = np.zeros(DD,dtype=np.float32)
for i in range(Dt.nx):
    dti[i,:,:] = dtn1[ind[i,0],:,:]*coe[i,0]+dtn1[ind[i,1],:,:]*coe[i,1]

In [9]:
eps = 1e-5
# normalize d0,dt,d0i,dti
d0 = (d0-np.mean(d0))/(np.std(d0)+eps)
d0i = (d0i-np.mean(d0i))/(np.std(d0i)+eps)
dt = (dt-np.mean(dt))/(np.std(dt)+eps)
dti = (dti-np.mean(dti))/(np.std(dti)+eps)

## Generate 3D patches

### load the reference CO2 mask

In [10]:
# readin CO2 mask
masks = np.fromfile(f'{mkfn}',dtype=np.float32)
masks = np.reshape(masks,DD)

### define slice labels and weights

In [11]:
# 2D vs 3D test: define x and y slices as interpreted in the 3D masks
inline_itp = np.array([60,80,100,120,130,140,150,160,180,200])
xline_itp = np.array([100,150,180,210,240,270,300,330,360,390,420,450,500])

In [12]:
noitp = 10
mask_itp = np.zeros_like(masks,dtype=np.float32)+noitp
for i,j in product(inline_itp,xline_itp):
    mask_itp[i,:,:] = masks[i,:,:]
    mask_itp[:,j,:] = masks[:,j,:]
weight = mask_itp!=noitp
mask_itp *= weight

In [13]:
# define wether to use slice interpretation
bool_sliceitp = True

In [14]:
# check the selected slices
r'''
for i in inline_itp:
    plt.imshow(mask_itp[i,:,:].T,vmin=0,vmax=1)
    plt.show()
for j in xline_itp:
    plt.imshow(mask_itp[:,j,:].T,vmin=0,vmax=1)
    plt.show()
'''

'\nfor i in inline_itp:\n    plt.imshow(mask_itp[i,:,:].T,vmin=0,vmax=1)\n    plt.show()\nfor j in xline_itp:\n    plt.imshow(mask_itp[:,j,:].T,vmin=0,vmax=1)\n    plt.show()\n'

In [15]:
# check the weight in 3D
#show3D(np.array(weight,dtype=np.float32))

### copy everything from fulllabel training dataset to sparselabel training dataset

In [16]:
fn = glob.glob(f'{outpath0}/*')
for fni in fn:
    shutil.copy(fni, f'{outpath}/')

In [17]:
with open(f'{outpath}/pm_info.json','r') as read_file:
    loaded_pm = json.loads(read_file.read())
    Ni,Nx,Nt = loaded_pm['patch_osize']
    rs = loaded_pm['patch_nsize']

### revise the masks and save corresponding weights

In [18]:
# sampling center range along x, y, and t directions
hNi = Ni//2
hNx = Nx//2
hNt = Nt//2

In [19]:
# 3d resize class
RS = resize3d(1,D=rs[0],H=rs[0],W=rs[0])

In [20]:
# load all training patch information
pf = pd.read_csv(f'{outpath}/patch_info_train.csv')
pid = pf['Ptch_id']
mid = pf['Mask_id']
ixt = pf['ct']
N = len(pid)

In [21]:
# revise the mask patches and add weight patches iteratively for training
hNi1 = Ni-hNi
hNx1 = Nx-hNx
hNt1 = Nt-hNt
for i in range(N):
    ixti = re.findall(r"\d+?\d*",ixt[i])
    Is = int(ixti[0])
    Xs = int(ixti[1])
    Ts = int(ixti[2])
    # slice the weighted mask
    if bool_sliceitp:
        M = np.array(mask_itp[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
        M = torch.tensor(M[::2,::2,::4]).unsqueeze(0).unsqueeze(0).numpy()
        #M = RS.resize(torch.tensor(M).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=64,H=64,W=64)
        W = np.array(weight[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
        W = torch.tensor(W[::2,::2,::4]).unsqueeze(0).unsqueeze(0).numpy()
        #W = RS.resize(torch.tensor(W).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=64,H=64,W=64)
    else:
        M = np.array(masks[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
        M = RS.resize(torch.tensor(M).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=64,H=64,W=64)        
    # save the revised mask and weight
    M.tofile(f'{outpath}/Mask_{mid[i]}.dat')
    if bool_sliceitp:
        W.tofile(f'{outpath}/Weight_{mid[i]}.dat')

In [22]:
# load all training patch information
pf = pd.read_csv(f'{outpath}/patch_info_valid.csv')
pid = pf['Ptch_id']
mid = pf['Mask_id']
ixt = pf['ct']
N = len(pid)

In [23]:
# revise the mask patches and add weight patches iteratively for validation
for i in range(N):
    ixti = re.findall(r"\d+?\d*",ixt[i])
    Is = int(ixti[0])
    Xs = int(ixti[1])
    Ts = int(ixti[2])
    # slice the weighted mask
    if bool_sliceitp:
        M = np.array(mask_itp[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
        M = torch.tensor(M[::2,::2,::4]).unsqueeze(0).unsqueeze(0).numpy()
        #M = RS.resize(torch.tensor(M).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=64,H=64,W=64)
        W = np.array(weight[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
        W = torch.tensor(W[::2,::2,::4]).unsqueeze(0).unsqueeze(0).numpy()
        #W = RS.resize(torch.tensor(W).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=64,H=64,W=64)
    else:
        M = np.array(masks[Is-hNi:Is+hNi1,Xs-hNx:Xs+hNx1,Ts-hNt:Ts+hNt1],dtype=np.float32)
        M = RS.resize(torch.tensor(M).unsqueeze(0).unsqueeze(0),NP=True) # 5D: (N=1,C=1,D=64,H=64,W=64)        
    # save the revised mask and weight
    M.tofile(f'{outpath}/Mask_{mid[i]}.dat')
    if bool_sliceitp:
        W.tofile(f'{outpath}/Weight_{mid[i]}.dat')